In [ ]:
import geopandas as gpd
import pandas as pd

# Read in data
from pathlib import Path

data_path = Path("..") / "data" / "feds_western_us_2013_af_postprocessed.parquet"

df = gpd.read_parquet(data_path)

print("COLUMNS:", df.columns)
print("CRS:", df.crs)
print(df.head())


FileNotFoundError: [Errno 2] Failed to open local file 'feds_western_us_2013_af_postprocessed.parquet'. Detail: [errno 2] No such file or directory

In [ ]:
#Find the longest duration fire and subset it for initial pipeline development
df_flat = df.reset_index()
max_area = df_flat.farea.idxmax()
max_fire_id = df_flat.loc[max_area,"fireID"]
hero_fire = df_flat.loc[df_flat.fireID == max_fire_id,:]
hero_fire.head(5)


In [ ]:
from geocube.api.core import make_geocube
hero_fire = hero_fire.rename_geometry("geometry")

# 2. Now run the exact same code as before
out_grid = make_geocube(
    vector_data=hero_fire,
    resolution=(-500, 500),
    measurements=["fireID"],
    output_crs="EPSG:9311",
    group_by="t"
)

print(out_grid)

## Okay, now I've got a simple grid for a single fire.. time to get weather data for it.

In [ ]:
# 1. Get the tight bounds of the fire (minx, miny, maxx, maxy)
minx, miny, maxx, maxy = hero_fire.total_bounds

# 2. Define a buffer (e.g., 10km = 10,000 meters)
BUFFER = 10000

# 3. Create the "Weather Box"
weather_box = [
    minx - BUFFER,  # West edge moves further West
    miny - BUFFER,  # South edge moves further South
    maxx + BUFFER,  # East edge moves further East
    maxy + BUFFER   # North edge moves further North
]

print(f"Fire Bounds:    {hero_fire.total_bounds}")
print(f"Weather Bounds: {weather_box}")

# 4. Define the Time Window
start_time = hero_fire.t.min()
end_time = hero_fire.t.max()

print(f"Need Weather from: {start_time} to {end_time}")